In [1]:
# Описание метода Гаусса

In [2]:
#Описание задания

класс - class Matrix. __str__(self, ); 

1) i := i + lambda * j
2) swap i, j
3) i := i * lambda, lambda != 0

def solve()
    self.forward_gauss()
    1 2 3
    2 3 4
    3 4 2

    1 1 1
    0 1 1
    0 0 0

    self.backward_gauss()


    1 0 0|3
    0 1 1|4


    x_1 = 3
    x_2 = 4 - x_3
    x_3

    self.get_solution()

    

class Solution:
    is_empty() -> bool
    num_free_vars() -> int
    get_some_solution(*args) -> Optional[List]:
        len(args) == self.num_free_vars()
        


    

    lambda x: [3, 4-x, x]
    lambda: [3, 4, 1]
    lambda: None

    
    


SyntaxError: unmatched ')' (3085257351.py, line 5)

In [1]:
import numpy as np
import sympy as sp
from typing import List, Union

In [31]:
m = np.array([
    [1, 2, 3, 4],
    [0, 0, 0 ,0],
    [5, 6, 7, 7]
])

In [34]:
np.all(m == 0, axis=1)

array([False,  True, False])

In [ ]:
def swap_rows(m, i, j):
    pass

def multiply_and_add_rows(m, from_row, to_row, scale):
    pass

def multiply_row(m, row, scale):
    pass

In [48]:
M = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

In [49]:
M[[i, i + j]]

array([[1, 2, 3],
       [4, 5, 6]])

In [46]:
i = 0
j = 1
M[[i, i + j]] = M[[i + j, i]]

In [47]:
M

array([[4, 5, 6],
       [1, 2, 3],
       [7, 8, 9]])

In [45]:
M[[i + j, i]]

array([[4, 5, 6],
       [1, 2, 3]])

In [75]:
class Matrix:
    def __init__(self, data: List[List[Union[int, float]]]):
        self.data = np.array(data)
        self.vars = sp.var(
            [f'x{i}' for i in range(1, np.array(self.data).shape[1])]
        )
        self.equations = [
            self.get_ith_equation(i)
            for i
            in range(len(data))
        ]
        self.triangular_matrix = None
        
    def __repr__(self):
        params = self.data[:, :-1]
        answers = self.data[:, -1]

        max_param = params.max().max()
        len_max_param = len(str(max_param))
        max_answer = answers.max().max()
        len_max_answer = len(str(max_answer))

        return '\n'.join(
            [f'|'.join(
                [''.join([f'{num:{len_max_param + 2}}' for num in row])] 
                + [f'{answer:{len_max_answer + 1}}']
            )
            for row, answer in zip(params, answers)]
        )

    def get_ith_equation(self, i):
        elements = [
            coef_i * x_i
            for coef_i, x_i
            in zip(self.data[i], self.vars)
        ]
        elements.append(-self.data[i, -1])
        return sum(elements)
    
    @staticmethod
    def swap_rows(M, i, j) -> None:
        M[[i, i + j]] = M[[i + j, i]]
        
    @staticmethod
    def multiply_and_add_rows(M, from_row, to_row, scale):
        M[to_row] += scale * M[from_row]
        
    @staticmethod
    def multiply_row(M, row, scale):
        M[row] *= scale
    
    def forward_gauss(self, M):
        # move all zeros to buttom of matrix
        M = np.concatenate((M[np.any(M != 0, axis=1)], M[np.all(M == 0, axis=1)]), axis=0)

        # iterate over matrix rows
        for i in range(M.shape[0]):

            # initialize row-swap iterator
            j = 1

            # select pivot value
            pivot = M[i][i]

            # find next non-zero leading coefficient
            while pivot == 0 and i + j < M.shape[0]:
                # perform row swap operation
                self.swap_rows(M, i, j)

                # incrememnt row-swap iterator
                j += 1

                # get new pivot
                pivot = M[i][i]

            # if pivot is zero, remaining rows are all zeros
            if pivot == 0:
                return M

            # extract row
            row = M[i]

            # get 1 along the diagonal
            self.multiply_row(M, i, 1 / pivot)

            # iterate over remaining rows
            for j in range(i + 1, M.shape[0]):
                # subtract current row from remaining rows
                self.multiply_and_add_rows(M, from_row=i, to_row=j, scale=-M[j][i])

        # return upper triangular matrix
        return M

    def backward_gauss(self, M):
        for i in range(M.shape[0] - 1, 0, -1):
            row = M[i]
            if not np.all(row == 0):
                main_coef_idx = row.nonzero()[0][0]

                # make zeros above the main coef
                for j in range(i - 1, -1, -1):
                    value_to_eliminate = M[j, main_coef_idx]
                    self.multiply_and_add_rows(M, from_row=i, to_row=j, scale=-value_to_eliminate)
        return M

    def get_solutions(self, M, syms):
        syms = syms[:]
        # symbolic variable index
        for i, row in reversed(list(enumerate(M))):
            # create symbolic equation
            eqn = -M[i][-1]
            for j in range(len(syms)):
                eqn += syms[j] * row[j]

            # solve symbolic expression and store variable
            try:
                syms[i] = sp.solve(eqn, syms[i])[0]
            except IndexError:
                # many solutions
                if np.all(self.triangular_matrix[-1] == 0):
                    remaining_equations = self.triangular_matrix[
                        ~np.all(self.triangular_matrix == 0, axis=1)
                    ]

                    solutions = (remaining_equations[:, :-1] @ m.vars - remaining_equations[:, -1]).tolist()
                    solutions = [
                        sp.Eq(var, equation) if var != equation
                        else var
                        for var, equation in zip(m.vars, sp.linsolve(solutions, self.vars).args[0])
                    ]
                    return sp.FiniteSet(*solutions)
                return []

        # return list of evaluated variables
        return syms
    
    def solve(self):
        self.triangular_matrix = self.forward_gauss(self.data.astype(np.float32))
        self.triangular_matrix = self.backward_gauss(self.triangular_matrix)
        result = self.get_solutions(self.triangular_matrix, self.vars)

        return result

In [76]:
data = [
    [1, 2, 3], 
    [3, 4, 50000]
]

m = Matrix(data)
m

  1  2|     3
  3  4| 50000

In [77]:
m.solve()

[49994.0000000000, -24995.5000000000]

In [78]:
m.equations[0]

x1 + 2*x2 - 3

In [79]:
data = [
    [1, 2, 3], 
    [3, 4, 50]
]

m = Matrix(data)
m

  1  2|  3
  3  4| 50

In [80]:
m.solve()

[44.0000000000000, -20.5000000000000]

In [81]:
m.triangular_matrix

array([[  1. ,   0. ,  44. ],
       [ -0. ,   1. , -20.5]], dtype=float32)

In [82]:
one_solution = [
    [
        [2, -3, 7],
        [4, 1, 2],
    ],
    [
        [1, 2, -1, 4],
        [2, -1, 3, 9],
        [3, 1, -2, 1],
    ],
    [
        [1, 2, -1, 3, 10],
        [2, 3, 1, -2, 5],
        [3, 0, -1, 4, 12],
        [4, 1, 1, 1, 8],
    ],
]

no_solutions = [
    [
        [2, -3, 7],
        [4, -6, 12],
    ],
    [
        [1, 2, -1, 4],
        [2, 4, -2, 8],
    ],
    [
        [1, 2, -1, 3, 10],
        [2, 3, 1, -2, 5],
        [3, -1, 4, 1, 12],
        [4, 0, 1, 1, 6],
    ],
]

infinite_solutions = [
    [
        [2, 3, 6],
        [4, 6, 12],
    ],
    [
        [1, -2, 5],
        [2, -4, 10],
    ],
    [
        [1, 2, -1, 3, 8],
        [2, 4, -2, 6, 16],
        [3, 6, -3, 9, 24],
        [4, 8, -4, 12, 32],
    ],
]

#### one solution

In [83]:
m = Matrix(one_solution[0])
m.solve()

[0.928571224212646, -1.71428585052490]

In [84]:
m.triangular_matrix

array([[ 1.       ,  0.       ,  0.9285712],
       [ 0.       ,  1.       , -1.7142859]], dtype=float32)

In [85]:
m = Matrix(one_solution[1])
m.solve()

[1.40000009536743, 2.79999995231628, 3.00000000000000]

In [86]:
m.triangular_matrix

array([[ 1.       ,  0.       ,  0.       ,  1.4000001],
       [-0.       ,  1.       ,  0.       ,  2.8      ],
       [-0.       , -0.       ,  1.       ,  3.       ]], dtype=float32)

In [87]:
m = Matrix(one_solution[2])
m.solve()

[2.28125000000000, 1.53125000000000, -3.15625000000000, 0.500000000000000]

#### no solutions

In [88]:
m = Matrix(no_solutions[0])
m.solve()

[]

In [89]:
m

  2 -3|  7
  4 -6| 12

In [90]:
m.triangular_matrix

array([[ 1. , -1.5, 10.5],
       [ 0. ,  0. , -2. ]], dtype=float32)

In [91]:
data = np.array([[1., 2., -1., 4.],
              [3., -1., 2., 7.],
              [0., 5., 4., 3.]], dtype=float)

In [94]:
Matrix(data).solve()

[2.60377359390259, 0.660377383232117, -0.0754717290401459]

#### many solutions

In [95]:
m = Matrix(infinite_solutions[0])
res = m.solve()
res

FiniteSet(x2, Eq(x1, 3.0 - 1.5*x2))

In [96]:
m.triangular_matrix

array([[1. , 1.5, 3. ],
       [0. , 0. , 0. ]], dtype=float32)

In [13]:
m = Matrix(infinite_solutions[1])
res = m.solve()
res

FiniteSet(x2, Eq(x1, 2.0*x2 + 5.0))

In [98]:
m = Matrix(infinite_solutions[2])
res = m.solve()
res

FiniteSet(x2, x3, x4, Eq(x1, -2.0*x2 + 1.0*x3 - 3.0*x4 + 8.0))